<a href="https://colab.research.google.com/github/fitditcorp4/research/blob/main/weekly_dataset02022023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# imports library
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas.plotting import autocorrelation_plot
from random import randrange
from statsmodels.tsa.seasonal import seasonal_decompose
#from pandas import datetime
from datetime import datetime
import numpy as np
from pandas import read_json
from pandas import read_csv
from numpy import array
import matplotlib.colors as colors
import matplotlib.cm as cm
from pylab import rcParams
import warnings
import itertools
from pandas import Series
# plt.style.use('fivethirtyeight')
plt.style.use('classic')
%matplotlib inline

In [3]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 1000)

In [4]:
#@title Default title text
df0 = pd.read_json('https://oilspillmonitor.ng/api/spill-data.php?dataset=nosdra&format=json')

In [5]:
df0

,id,status,company,incidentnumber,incidentdate,contaminant,estimatedquantity,sitelocationname,spillareahabitat,lastupdatedby,cause,updatefor,latitude,longitude,statesaffected,attachments,jivdate,lga,zonaloffice,certificatedate,certificatenumber,formadate,reportdate,spillstopdate,initialcontainmentmeasures,descriptionofimpact,formbdate,formcdate,jivpresent,typeoffacility,cleanupcompleteddate,cleanupmethods,cleanupdate,quantityrecovered,postcleanupinspectiondate,finalsamplingdate,estimatedspillarea,remediationstart,remediationtype,remediationend,postimpactassessmentdate,finallabresultsdate
0,2,confirmed,ADDAX,HSE/OBO/0611/101,2006-11-23,cr,225,Subsea Pipeline @ Mimbo Platform (OML123),of,NOSDRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,confirmed,ADDAX,HSE/OBO/0612/108,2006-12-18,cr,0.5,EBNE04HT @ Ebughu Platform (OML 123),of,NOSDRA,eqf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,confirmed,ADDAX,HSE/OBO/0612/110,2006-12-27,cr,25,South East of BOGI Platform,of,NOSDRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,confirmed,ADDAX,HSE/OBO/0706/166,2007-05-14,cr,2,Adanga Platform,of,NOSDRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,confirmed,ADDAX,HSE/OBO/0708/201,2007-08-16,gs,NaN,"Izombe Well #10LS 4"" Flowline",la,NOSDRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16415,268076,confirmed,SEEPCO,SEEPCO/HSE/NOSDRA/01/23,2023-01-13,cr,0.5,ANIEZE JUNCTION TO ANIEZE B AT ASHAKA OKPAI,la,2023-01-25 WA003,ome,NaN,5.7735417,6.6189853,DE,"[{""url"":""data/attachments/t185eab17d91/1.jpg"",...",2023-01-19,Ndokwa-East,wa,NaN,NaN,NaN,2023-01-14,NaN,other:none,"Soil and vegetation were impacted, the spill w...",NaN,NaN,"nosdra,cmpny,cmmnt",pl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16416,268074,confirmed,SEPLAT,SEPLAT/WA/OS/01/2023,2023-01-16,no,NaN,6'' Okpohuru - Oben Bulkline 4 at Ugbevwe Comm...,la,2023-01-26 WA003,sab,268074.0,5.9216833,5.8465972,DE,"[{""url"":""data/attachments/t185e92c102d/seplat ...",2023-01-23,Ethiope-West,wa,NaN,NaN,NaN,2023-01-16,NaN,NaN,4'' Valve attached to a 4'' Galvanised pipe of...,NaN,NaN,"nosdra,cmmnt,cmpny,dpr",other:Bulkline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16417,268086,confirmed,SEPLAT,SEPLAT/WA/OS/13G/2022,2022-09-05,cr,0.63,20'' AMUKPE - ESCRAVOS PIPELINE ROW @ UBAGBORO,sw,2023-01-26 WA003,sab,NaN,5.8316667,5.5225000,DE,"[{""url"":""data/attachments/t185ef0e313f/IMG_202...",2023-01-24,Warri-South,wa,NaN,NaN,NaN,2022-09-06,NaN,other:none,"Vegetation and surface water were impacted, th...",NaN,NaN,"nosdra,dpr,mnstry,cmpny,cmmnt",pl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16418,268092,confirmed,Midwestern,UPIL/001/2023,2023-01-23,cr,0.5,12'' MIDWESTERN OIL AND GAS PIPELINE AT EBENDO,la,2023-01-28 WA003,sab,NaN,5.6259517,6.3087708,DE,"[{""url"":""data/attachments/t185f81a4698/1.jpg"",...",2023-01-26,Isoko-North,wa,NaN,NaN,NaN,2023-01-23,NaN,other:none,"Soil was impacted, the spill was within the co...",NaN,NaN,"nosdra,cmpny,cmmnt",pl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
